#

Needed per subject:

- *hardi_2mm_2b_as.rec & hardi_2mm_2b_ps.rec --> parec2nii (will give necessary bval/bves files and all 33 directions in one files)
- *hardi_2mm_2b_as_bvalue1_diffori33.nii & *hardi_2mm_2b_ps_bvalue1_diffori33.nii as b0 images necessary for topup (fieldmap correction)
- acqparams.txt = describing the phase encoding direction and echo time for TOPUP | checked the scan sequence, should be (according to ChatGPT given the info of the file) 
0  1  0  0.0215
0 -1  0  0.0215

- 

1. fslmerge -t b0_all sn_*_as_bvalue1_diffori33.nii sn_*_ps_bvalue1_diffori33.nii
2. topup --imain=b0_all --datain=acqparams.txt --config=b02b0.cnf --subsamp=1 --out=topup_results --iout=corrected_b0
3. bet corrected_b0.nii.gz brain -m -f 0.2 (ChatGPT, contradicts itself, better with b0 or T1 ?! -- ives error in next steps with T1 cause of mismatch in voxel numbers...), therefore: bet corrected_b0 nodif_brain -m
4. fslmerge -t dwi_all.nii.gz  *hardi_2mm_2b_as.nii *hardi_2mm_2b_ps.nii
5. paste -d' ' dwi_as.bval dwi_ps.bval > dwi_all.bval
6. paste -d' ' dwi_as.bvec dwi_ps.bvec > dwi_all.bvec

### A. 
7. Eddy:
eddy --imain=dwi_all --mask=nodif_brain_mask \                                              
    --acqp=acqparams.txt --index=index.txt \
    --bvecs=dwi_all.bvecs --bvals=dwi_all.bval \
    --topup=topup_results --out=eddy_corrected
8. dtifit: 
dtifit --data=eddy_corrected \                                                              
       --mask=nodif_brain_mask \
       --bvecs=eddy_corrected.eddy_rotated_bvecs \
       --bvals=dwi_all.bval \
       --out=dtifit_results
#### B. without Eddy:
7. applytopup --imain=dwi_all --topup=topup_results --datain=acqparams.txt --inindex=1 --method=jac --out=dwi_topup_corrected
8. dtifit --data=dwi_topup_corrected --mask=nodif_brain_mask --bvecs=dwi_all.bvecs --bvals=dwi_all.bval --out=dtifit_topup_corrected_results


https://fsl.fmrib.ox.ac.uk/fslcourse/2019_Beijing/lectures/FDT/fdt1.html

## Succesfull FA image generation:

1.	Transform the .par files to nifti while also creating the necessary .bval & .bvec files (parrec2nii --bvs *_ps.par)
2.	Merge all into on large 4D file (66 in dim4):  fslmerge -t dwi_all.nii.gz *_as.nii *_ps.nii
    a.	Also merge bvec &bval files: paste -d' ' dwi_as.bval dwi_ps.bval > dwi_all.bval (same for .bvecs)
3.	Merge b0 files: fslmerge -t b0_all sn_*_as_bvalue1_diffori33.nii sn_*_ps_bvalue1_diffori33.nii
4.	Get distortion corrections: topup --imain=b0_all --datain=acqparams.txt --config=b02b0.cnf --subsamp=1 --out=topup_results --iout=corrected_b0
5.	Apply topup results: applytopup --imain=dwi_all --topup=topup_results --datain=acqparams.txt --inindex=1 --method=jac --out=dwi_topup_corrected
6.	Get brain mask from mean dti image (the version where I generate the brain mask from corrected_b0 seems to misalign) :
    a.	fslmaths dwi_topup_corrected.nii.gz -Tmean dwi_topup_corrected_mean
    b.	bet dwi_topup_corrected_mean.nii.gz mask_dwi_topup_corrected_mean -m -f 0.2
7.	Fit diffusion tensor model at each voxel: dtifit --data=dwi_topup_corrected --mask=mask_dwi_topup_corrected_mean_mask --bvecs=dwi_all.bvecs --bvals=dwi_all.bvals --out=dtifit _meanmask_results


/usr/local/fsl/bin/dtifit --data=/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/tryout_DTI/sub-02/dwi_all.nii.gz --out=/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/tryout_DTI/sub-02/dtifit_results_GUI --mask=/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/tryout_DTI/sub-02/nodif_brain_mask.nii.gz --bvecs=/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/tryout_DTI/sub-01/bvecs_all_T.txt --bvals=/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/tryout_DTI/sub-01/bvals_all_T.txt --wls


what worked: 
dtifit --data=dwi_topup_corrected --mask=mask_dwi_topup_corrected_mean --bvecs=dwi_all.bvecs --bvals=dwi_all.bval --out=dtifit_meammask_results

## Tract-Based Spatial Statistics (TBSS)
https://andysbrainbook.readthedocs.io/en/stable/TBSS/TBSS_Overview.html

--> wants all individual 


In [ ]:
# Tranpose files | not needed !

fn_in = '/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/tryout_DTI/sub-01/bvals_all.txt'
fn_out = '/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/tryout_DTI/sub-01/bvals_all_T.txt'
# Read the file
with open(fn_in, "r") as file:
    lines = [line.strip().split() for line in file]

# Transpose the data
transposed = list(zip(*lines))

# Write the transposed data to a new file
with open(fn_out, "w") as file:
    for row in transposed:
        file.write(" ".join(row) + "\n")
